In [177]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from scipy.optimize import minimize
from scipy.optimize import differential_evolution

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings('ignore')


In [178]:
def categorizar(df):
    
    df['classe_bonus_bin'] = df['classe_bonus'].apply(lambda x: '00'  if x == 0 else 'Demais')   
    
    df['MD_sexo_bin'] = df['MD_sexo'].map({'Masculino': 'Masculino', 
                                 'Feminino': 'Feminino', 
                                  'ERRO': 'Masculino'}).astype(object)
    
    df['MD_sexo_2bin'] = df['MD_sexo'].map({'Masculino': 1, 
                                 'Feminino': 0, 
                                  'ERRO': 1}).astype(object)


    df['fx_ir_12m_bin'] = df['fx_ir_12m'].apply(lambda x: '01_00_20' if x == '01 - 00-<=020' else
                                                          '02_20_40' if x == '02 - 20-<=040' else
                                                          '03_40_60' if x == '03 - 40-<=060' else
                                                          '04_60_80' if x == '04 - 60-<=080' else
                                                          '05_80_100' if x == '05 - 80-<=100' else
                                                          '02_20_40' if x == 'NI'           else 'ERROR')

  
    df['MD_comb_bin'] = df['MD_comb'].apply(lambda x: 'Gasolina' if x == 'Gasolina'  else 'Demais')

    df['canal_bin'] =  df['canal'].apply(lambda x: 'Corretor_Mais' if x == 'Corretor Mais' else 
                                                   'Corretor_Mais' if x == 'CorretorMais'
                                                                   else    'Demais')
   
    df['per_com_bin'] = df['per_com'].apply(lambda x: 'p_1_10' if x < 10 else
                                                      'p_10_35' if x >= 10 else 'ERROR')    



    df['idade_ult_RNS_bin'] = df['idade_ult_RNS'].apply(lambda x:     'Sem_Sinistro' if x == 'Sem Sinistro' else
                                                                      '01_05_anos' if x == '01 ano' else
                                                                      '01_05_anos' if x == '02 anos' else
                                                                      '01_05_anos' if x == '03 anos' else
                                                                      '01_05_anos' if x == '04 anos' else
                                                                      '01_05_anos' if x == '05 anos' else '06_inf')


    df['UF_bin'] = df['UF'].apply(lambda x:     'AL_CE_RS'  if x == 'AL' else
                                                'AL_CE_RS'  if x == 'CE' else
                                                'AL_CE_RS'  if x == 'RS' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'AP' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'DF' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'ES' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PA' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PB' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'RN' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'SC' else 
                                                'GO_MA_MG'             if x == 'GO' else 
                                                'GO_MA_MG'             if x == 'MA' else 
                                                'GO_MA_MG'             if x == 'MG' else 
                                                'MT_PR'                if x == 'MT' else
                                                'MT_PR'                if x == 'PR' else 
                                                'RJ'                   if x == 'RJ' else  
                                                'Demais')

    df['md_tipo_renov_ant_bin'] = df['Renovacao_ant'].apply(lambda x:     '1_SeguroNovo'       if x == '1 - Seguro Novo'  else
                                                                          '2_RenovPropria'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                          '2_RenovPropria'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                          '3_RenovCongenere'   if x == '4 - Renov Congenere sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '5 - Renov Congenere com sinistro' else
                                                                          '3_RenovCongenere'   if x == '6 - Renov BB sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '7 - Renov BB com sinistro' else  
                                                                          '2_RenovPropria')
  
    df['MD_tipo_renov_bin'] = df['MD_tipo_renov'].apply(lambda x:          '2_RenovPropria_sem_sin'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                           '3_RenovPropria_com_sin'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                           'Error')
   
     
    df['Cod_End_bin'] = df['Cod_End'].apply(lambda x: '13509'  if x == 13509 else
                                                      '13509'  if x == 50007 else
                                                      '13509'  if x == 50100 else 'Demais')   

    df['MD_capac_bin'] = df['MD_capac'].apply(lambda x: '5'  if x == 5  else 'Demais')  



    df['fator_ajuste_bin'] = df['MD_fator_ajuste'].apply(lambda x: 'f_80_101'    if x < 101 else
                                                                   'f_101_120'   if x >= 101  else 
                                                                   'f_80_101')   
    
    return df

In [179]:
def demand_model(df):
    import pandas as pd
    import numpy as np 
    P   = df['premio']
    X2  = np.array(df['GBM_mercado'])
    X3  = np.array(df['Premio_IC_100'])
    X4  = np.array(df['premio_anterior'])
    X5  = np.array(df['MD_tempo_casa_2'])
    X6  = np.array(df['per_com'])
    X7  = np.array(df['per_com_bin_p_1_10'])
    X1  = np.array(df['is_casco_ofer'])
    X8  = np.array(df['canal_bin_Corretor_Mais'])
    X9  = np.array(df['Cod_End_bin_13509'])
    X10 = np.array(df['fator_ajuste_bin_f_101_120'])
    X11 = np.array(df['classe_bonus_bin_00'])
    X12 = np.array(df['UF_bin_AL_CE_RS'])
    X13 = np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
    X14 = np.array(df['UF_bin_GO_MA_MG'])
    X15 = np.array(df['UF_bin_MT_PR'])
    X16 = np.array(df['UF_bin_RJ'])
    X17 = np.array(df['MD_comb_bin_Gasolina'])
    X18 = np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
    X19 = np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
    X20 = np.array(df['fx_ir_12m_bin_01_00_20'])
    X21 = np.array(df['fx_ir_12m_bin_03_40_60'])
    X22 = np.array(df['fx_ir_12m_bin_04_60_80'])
    X23 = np.array(df['fx_ir_12m_bin_05_80_100'])
    X24 = np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
    X25 = np.array(df['idade_ult_RNS_bin_01_05_anos'])
    X26 = np.array(df['idade_ult_RNS_bin_06_inf'])

    def demanda(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
        D=np.exp(1.3356 -0.2576 * np.log(P)  + 1.3647 * np.divide(P, X1)-0.8782 * np.log(np.divide(P, X2))-0.795 * np.log(np.divide(P, X3))-1.6731 * np.log((np.divide(P, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
        return D/(1+D)


    #df['Renewal_Demand'] = demanda(P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26)
    
    return  demanda(P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26)
  

In [180]:
def relatorio(df):
    import pandas as pd
    import numpy as np 
    df["per_com_pct"] = df['per_com']/100
    df["premio_prob"] = df["Renewal_Demand"]*df["premio"]
    df["premio_anterior_prob"] = df["Renewal_Demand"]*df["premio_anterior"]
    df["Act_cost_prob"] = df["Renewal_Demand"]*df["Act_cost"]                                                                
    df["comissao_valor"] = df["per_com_pct"] *df["premio"]
    df["comissao_valor_prob"] = df["per_com_pct"] *df["premio_prob"]



    apolice_ofertada    = np.round(df.shape[0],0)
    apolice_estimada    = np.round(np.sum(df["Renewal_Demand"]),0)
    premio_ofertado     = np.mean(df["premio"])
    retencao_estimada   = np.mean(df["Renewal_Demand"])
    premio_esperado     = np.sum(df["premio_prob"])/np.sum(df["Renewal_Demand"])
    risco_ofertado      = np.mean(df["Act_cost"])
    risco_esperado      = np.sum(df["Act_cost_prob"])/np.sum(df["Renewal_Demand"])
    sp_ofertado         = np.sum(df["Act_cost"])/np.sum(df["premio"])
    sp_esperado         = np.sum(df["Act_cost_prob"])/np.sum(df["premio_prob"])
    comis_ofertado      = np.sum(df["comissao_valor"])/np.sum(df["premio"])
    comis_esperado      = np.sum(df["comissao_valor_prob"])/np.sum(df["premio_prob"])
    premio_anterior     = np.mean(df["premio_anterior"])
    premio_anterior_w   = np.sum(df["premio_anterior_prob"])/np.sum(df["Renewal_Demand"])
    ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
    ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749
    premio_anterior   = np.mean(df_lote["premio_anterior"])
    premio_anterior_w   = np.sum(df_lote["premio_anterior_prob"])/np.sum(df_lote["Renewal_Demand"])


    tabela = pd.DataFrame([apolice_ofertada,apolice_estimada,
                            retencao_estimada,
                            premio_ofertado,premio_esperado,
                            premio_anterior,premio_anterior_w,
                            risco_ofertado,risco_esperado,
                            comis_ofertado, comis_esperado,
                            sp_ofertado,sp_esperado,
                            ic_ofertado,ic_esperado      
                            ])


    tabela = tabela.rename(columns = {0 : 'Total'})

    tabela.index = ['Apolices Ofertadas:', 
                    'Apolices Renovadas:',
                    '%Retenção:',
                    'Premio Ofertado:',
                    'Premio Renovado:',  
                    'Premio Anterior Ofertado:',
                    'Premio Anterior Renovado:',
                    'Risco Ofertado', 
                    'Risco Renovado', 
                    '%Comissão Ofertada:',
                    '%Comissão Renovada:',
                    '%SP Ofertada:', 
                    '%SP Renovada:',
                    '%IC Ofertado:', 
                    '%IC Renovado:']

    return tabela

In [181]:
def relatorio_otim(df):
    import pandas as pd
    import numpy as np 
    
    df["retencao_otimizada"] = df["retencao_otimizada"]
    df["precos_otimizados"] = df["precos_otimizados"]
    
    df["per_com_pct"] = df['per_com']/100
    df["premio_prob"] = df["retencao_otimizada"]*df["precos_otimizados"]
    df["premio_anterior_prob"] = df["retencao_otimizada"]*df["premio_anterior"]
    df["Act_cost_prob"] = df["retencao_otimizada"]*df["Act_cost"]                                                                
    df["comissao_valor"] = df["per_com_pct"] *df["precos_otimizados"]
    df["comissao_valor_prob"] = df["per_com_pct"] *df["premio_prob"]



    apolice_ofertada    = np.round(df.shape[0],0)
    apolice_estimada    = np.round(np.sum(df["retencao_otimizada"]),0)
    premio_ofertado     = np.mean(df["precos_otimizados"])
    retencao_estimada   = np.mean(df["retencao_otimizada"])
    premio_esperado     = np.sum(df["premio_prob"])/np.sum(df["retencao_otimizada"])
    risco_ofertado      = np.mean(df["Act_cost"])
    risco_esperado      = np.sum(df["Act_cost_prob"])/np.sum(df["retencao_otimizada"])
    sp_ofertado         = np.sum(df["Act_cost"])/np.sum(df["precos_otimizados"])
    sp_esperado         = np.sum(df["Act_cost_prob"])/np.sum(df["premio_prob"])
    comis_ofertado      = np.sum(df["comissao_valor"])/np.sum(df["precos_otimizados"])
    comis_esperado      = np.sum(df["comissao_valor_prob"])/np.sum(df["premio_prob"])
    premio_anterior     = np.mean(df["premio_anterior"])
    premio_anterior_w   = np.sum(df["premio_anterior_prob"])/np.sum(df["retencao_otimizada"])
    ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
    ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749
    premio_anterior   = np.mean(df_lote["premio_anterior"])
    premio_anterior_w   = np.sum(df_lote["premio_anterior_prob"])/np.sum(df_lote["retencao_otimizada"])


    tabela = pd.DataFrame([apolice_ofertada,apolice_estimada,
                            retencao_estimada,
                            premio_ofertado,premio_esperado,
                            premio_anterior,premio_anterior_w,
                            risco_ofertado,risco_esperado,
                            comis_ofertado, comis_esperado,
                            sp_ofertado,sp_esperado,
                            ic_ofertado,ic_esperado      
                            ])


    tabela = tabela.rename(columns = {0 : 'Total'})

    tabela.index = ['Apolices Ofertadas:', 
                    'Apolices Renovadas:',
                    '%Retenção:',
                    'Premio Ofertado:',
                    'Premio Renovado:',  
                    'Premio Anterior Ofertado:',
                    'Premio Anterior Renovado:',
                    'Risco Ofertado', 
                    'Risco Renovado', 
                    '%Comissão Ofertada:',
                    '%Comissão Renovada:',
                    '%SP Ofertada:', 
                    '%SP Renovada:',
                    '%IC Ofertado:', 
                    '%IC Renovado:']

    return tabela

## Base Para escorar e otimizar

In [182]:
df_lote_102024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/AJUSTADO Base Case Earnix - Grupo Segmentacao.csv",
                             dtype={'chave': 'string'},
                     encoding='latin-1',
                     delimiter='|')

df_lote_102024['GBM_mercado']  = df_lote_102024['GBM_mercado']/(1-df_lote_102024['per_com']/100)

In [183]:
df_lote_escorar=df_lote_102024.copy()

In [184]:
df_lote_escorar["premio"] = df_lote_escorar["Premio_oferecido_com_batente"]

In [185]:
df_lote_escorar.rename(columns={'Pr_anterior_TI':'premio_anterior', 
                                   'cod_end': 'Cod_End',
                                   'md_capac': 'MD_capac',
                                   'Classe_bonus':'classe_bonus',
                                   'fator_ajuste': 'MD_fator_ajuste'},inplace=True)

In [186]:
df_lote = categorizar(df_lote_escorar)

In [187]:
df_lote.shape

(1473, 144)

In [188]:
df_cat = df_lote[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
df_cat_glm=pd.get_dummies(df_cat)
df_lote=pd.concat([df_lote,df_cat_glm],axis=1)

In [189]:
df_lote.drop(['Renewal Demand','Premium Parameter'],axis='columns', inplace=True)

In [190]:
df_lote['Renewal_Demand'] = demand_model(df_lote)

In [191]:
relatorio(df_lote)

,Total
Apolices Ofertadas:,1473.000000
Apolices Renovadas:,756.000000
%Retenção:,0.513468
Premio Ofertado:,3063.388364
Premio Renovado:,2381.388210
Premio Anterior Ofertado:,2893.832077
Premio Anterior Renovado:,2614.777661
Risco Ofertado,1767.808781
Risco Renovado,1345.266898
%Comissão Ofertada:,0.159572


In [192]:
df_lote['Renewal_Demand'].mean()

0.513467580404544

## Otimização Teste 

In [47]:
#print(df_lote_otim['elasticidade'].value_counts(ascending=False))

In [193]:
#df_lote_otim=df_mod.copy()
df_lote_otim_atu = df_lote.query('Amostra_2 == "Segmentacao"')
df_lote_otim_atu.shape

(1473, 180)

In [194]:
df_lote_otim = df_lote_otim_atu
df_lote_otim.shape

(1473, 180)

In [195]:
df_lote_otim['Renewal_Demand'].mean()

0.513467580404544

#########################################

In [198]:
%%time
######
import sympy as sp
from scipy.optimize import fsolve

df = df_lote_otim


P=(df['premio'])
X1=np.array(df['is_casco_ofer'])
X2=np.array(df['GBM_mercado'])
X3=np.array(df['Premio_IC_100'])
X4=np.array(df['premio_anterior'])
X5=np.array(df['MD_tempo_casa_2'])
X6=np.array(df['per_com'])
X7=np.array(df['per_com_bin_p_1_10'])
X8=np.array(df['canal_bin_Corretor_Mais'])
X9=np.array(df['Cod_End_bin_13509'])
X10=np.array(df['fator_ajuste_bin_f_101_120'])
X11=np.array(df['classe_bonus_bin_00'])
X12=np.array(df['UF_bin_AL_CE_RS'])
X13=np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
X14=np.array(df['UF_bin_GO_MA_MG'])
X15=np.array(df['UF_bin_MT_PR'])
X16=np.array(df['UF_bin_RJ'])
X17=np.array(df['MD_comb_bin_Gasolina'])
X18=np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
X19=np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
X20=np.array(df['fx_ir_12m_bin_01_00_20'])
X21=np.array(df['fx_ir_12m_bin_03_40_60'])
X22=np.array(df['fx_ir_12m_bin_04_60_80'])
X23=np.array(df['fx_ir_12m_bin_05_80_100'])
X24=np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
X25=np.array(df['idade_ult_RNS_bin_01_05_anos'])
X26=np.array(df['idade_ult_RNS_bin_06_inf'])
X26=np.array(df['idade_ult_RNS_bin_06_inf'])


def calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
    D=np.exp(1.3356 -0.2576 * np.log(P)  + 1.3647 * np.divide(P, X1)-0.8782 * np.log(np.divide(P, X2))-0.795 * np.log(np.divide(P, X3))-1.6731 * np.log((np.divide(P, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
    return D/(1+D)


# Definir a função objetivo que maximiza a margem esperada otimizada 
def objetivo(P):
    
    df['precos']= P *1.23
        
    # Calcular a retenção otimizada com os preços dados
    df['retencao_otimizada1'] = calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)
    
    # Calcular a margem esperada
    df['margem_esperada_otimizada'] =   df['retencao_otimizada1']*(df['precos'] - df['Act_cost'])
    
  
    margem_esperada_total = df['margem_esperada_otimizada'].sum() 

    # Maximizar a margem 
    return -margem_esperada_total

# Definir os limites para os preços otimizado
nome_variaveis = ['LB','UB']
limites=np.array(df.loc[:,nome_variaveis])

# Executar a otimização 
resultado = differential_evolution(objetivo, bounds=limites, strategy='best1bin', maxiter=3000, popsize=11,seed=df.shape[0])

# Resultados
print(resultado.message) 
print("Tamanho da Base:", df.shape[0])
precos_otimizados = resultado.x
margem_esperada_total_otimizada = -resultado.fun


Optimization terminated successfully.
Tamanho da Base: 1473
CPU times: total: 1min 45s
Wall time: 2min 9s


In [199]:
print(df['precos'].mean()),
print(np.mean(precos_otimizados)),
print(np.mean(-resultado.fun))

3761.9398461390774
3065.5006656094965
1206655.845539067


In [200]:
df['precos_otimizados'] = precos_otimizados

df['retencao_otimizada'] = calcular_retencao(df['precos_otimizados'],X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)

df['margem_esperada_base_case'] = df['Renewal_Demand']*(df['premio']-df['Act_cost'])

margem_esperada_total_base_case=df['margem_esperada_base_case'].sum()
  
    
delta_q = (df['retencao_otimizada']-df['Renewal_Demand'])
delta_p = (df['precos_otimizados']-df['premio'])
   
delta_q_b = ((df['retencao_otimizada']+df['Renewal_Demand'])/2)
delta_p_b = ((df['precos_otimizados']+df['premio'])/2)

df["elasticidade"] = (delta_q/delta_p)*(delta_p_b/delta_q_b)
df["elasticidade_2"] = (df['retencao_otimizada']/df['Renewal_Demand']-1)/(df['precos_otimizados']/df['premio']-1)  

df["elasticidade"] = df["elasticidade"].fillna(-1)
df["elasticidade_2"] = df["elasticidade_2"].fillna(-1)

df['elasticidade_bin'] = pd.cut(x=df['elasticidade'], bins=[-3.5,-2.5,-2,-1.5,-1,0])  

df.replace([np.inf, -np.inf], -1, inplace=True)



#df["SP_ofer_python"] = df["Act_cost"]/df["precos_otimizados"]
#df["IC_ofer_python"] = df["per_com_pct"] + df["SP_ofer_python"] +  0.2749
#df["SP_ofer_eranix"] = df["Act_cost"]/df["Premium Parameter"]
#df["IC_ofer_eranix"] = df["SP_ofer_eranix"] + df["per_com_pct"]+  0.2749
#df["varia_prem_anterior_py"] = df["precos_otimizados"]/df["premio_anterior"]
#df["varia_prem_anterior_earnix"] = df["Premium Parameter"]/df["premio_anterior"]
#df["varia_prem_ofer_py"] = df["precos_otimizados"]/df["premio"]
#df["varia_prem_ofer_earnix"] = df["Premium Parameter"]/df["premio"]
#df["varia_prem_py_earnix"] = df["precos_otimizados"]/df["Premium Parameter"]


#df['IC_ofer_python_bin'] = pd.cut(x=df['IC_ofer_python'], bins=[0.7,0.95,0.99,1,1.03,1.9])  
#df['IC_ofer_eranix'] = pd.cut(x=df['IC_ofer_eranix'], bins=[0.7,0.95,0.99,1,1.03,1.9]) 


# Exibir os resultados
print("Margem Esperada Total Base Case:", margem_esperada_total_base_case) 
print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada) 
print("Retenção Média Base Case:", df['Renewal_Demand'].mean())
print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())
print("Premio Medio Otimizado:", df['precos_otimizados'].mean())
print("Premio Medio Base Case:", df['premio'].mean())
print("Elasticidade:", df['elasticidade'].mean())
print("Elasticidade2:", df['elasticidade_2'].mean())

Margem Esperada Total Base Case: 783657.6571234912
Margem Esperada Total Otimizada: 1206655.845539067
Retenção Média Base Case: 0.513467580404544
Retenção Média Otimizada: 0.5094637098953982
Premio Medio Otimizado: 3065.5006656094965
Premio Medio Base Case: 3063.3883643280246
Elasticidade: -1.6137229420245336
Elasticidade2: -1.6252969017565804


In [201]:
df['retencao_otimizada'].mean()

0.5094637098953982

In [202]:
df['Renewal_Demand'].mean()

0.513467580404544

In [203]:
df['precos_otimizados'].mean()

3065.5006656094965

In [40]:
#relatorio_otim(df)

#criando a função

In [131]:
df_relatorio_otim = df.copy()


df_relatorio_otim['precos_otimizados'] = df_relatorio_otim['precos_otimizados']


df_relatorio_otim['batente_otim']=np.round(df_relatorio_otim['precos_otimizados']/df_relatorio_otim['premio'],2)
df_relatorio_otim['batente_otim_prem_ant']=np.round(df_relatorio_otim['precos_otimizados']/df_relatorio_otim['premio_anterior'],2)



df_relatorio_otim['retencao_otimizada'] = calcular_retencao(df_relatorio_otim['precos_otimizados'],X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)


df_relatorio_otim["per_com_pct"] = df_relatorio_otim['per_com']/100
df_relatorio_otim["premio_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["Act_cost_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["Act_cost"]                                                                
df_relatorio_otim["comissao_valor"] = df_relatorio_otim["per_com_pct"] *df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["comissao_valor_prob"] = df_relatorio_otim["per_com_pct"]*df_relatorio_otim["premio_prob"]

df_relatorio_otim["premio_anterior_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["premio_anterior"]

comis_ofertado    = np.sum(df_relatorio_otim["comissao_valor"])/np.sum(df_relatorio_otim["precos_otimizados"])
comis_esperado    = np.sum(df_relatorio_otim["comissao_valor_prob"])/np.sum(df_relatorio_otim["premio_prob"])

premio_anterior_ofer    = np.mean(df_relatorio_otim["premio_anterior"])
premio_anterior_w   = np.sum(df_relatorio_otim["premio_anterior_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])


apolice_ofertada  = np.round(df_relatorio_otim.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio_otim["retencao_otimizada"]),0)
premio_ofertado   = np.mean(df_relatorio_otim["precos_otimizados"])
retencao_estimada = np.mean(df_relatorio_otim["retencao_otimizada"])
premio_esperado   = np.sum(df_relatorio_otim["premio_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
risco_ofertado    = np.mean(df_relatorio_otim["Act_cost"])
risco_esperado    = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
sp_ofertado       = np.sum(df_relatorio_otim["Act_cost"])/np.sum(df_relatorio_otim["precos_otimizados"])
sp_esperado       = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["premio_prob"])
comis_ofertado    = np.sum(df_relatorio_otim["comissao_valor"])/np.sum(df_relatorio_otim["precos_otimizados"])
comis_esperado    = np.sum(df_relatorio_otim["comissao_valor_prob"])/np.sum(df_relatorio_otim["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749


print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))

print('comis_ofertado" : %s'  % np.round(comis_ofertado*100,2))
print('comis_esperado" : %s'  % np.round(comis_esperado*100,2))

print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

print('premio_anterior_ofer" : %s'  %np.round( premio_anterior_ofer,2))
print('premio_anterior_w" : %s'  %np.round( premio_anterior_w,2))

Apolices Ofertadas: 1473
Apolices Renovadas: 750.0
premio_ofertado: 3065.5
retencao_estimada : 50.95
premio_esperado" : 2408.03
risco_ofertado" : 1767.81
risco_esperado" : 1353.95
comis_ofertado" : 15.94
comis_esperado" : 15.83
sp_ofertado" : 57.67
sp_esperado" : 56.23
ic_ofertado" : 101.1
ic_esperado" : 99.55
premio_anterior_ofer" : 2893.83
premio_anterior_w" : 2615.18


In [132]:
df_relatorio_otim['precos_otimizados'].mean()

3065.5006656094965

In [133]:
df_relatorio_otim['batente_otim_prem_ant'].describe()

count    1473.000000
mean        1.068853
std         0.354766
min         0.520000
25%         0.780000
50%         0.950000
75%         1.270000
max         1.800000
Name: batente_otim_prem_ant, dtype: float64

In [134]:
df_relatorio_otim['batente_otim'].describe()

count    1473.000000
mean        1.006572
std         0.051566
min         0.900000
25%         0.970000
50%         1.000000
75%         1.050000
max         1.100000
Name: batente_otim, dtype: float64

In [135]:
analitico_202410_ajustado=df_relatorio_otim
analitico_202410_ajustado.drop(['Renewal_Demand','retencao_otimizada1'],axis='columns', inplace=True)
analitico_202410_ajustado.to_csv(r'analitico_202410_ajustado.csv', sep='|' ,header='true')